In [2]:
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

In [3]:
# Define the object we use
import matplotlib.dates as dates
from datetime import datetime

class geminiData:
    GainLoss = 0
    Variance = 0
    Volume   = 0
    Hour     = 0
    Day      = 0
    Date     = 0;
    OpenPrice = 0
    ClosePrice = 0

geminiDatas = []

class preBubble:
    GainLoss = 0
    Variance = 0
    Volume   = 0
    Hour     = 0
    Day      = 0
    Date     = 0;

preBubble = []

geminiX = np.empty([0,1])

In [4]:
# reference to this https://realpython.com/python-csv/
import csv
from numpy import empty
import numpy as np

with open ('GEMINI_BTC.csv') as csv_file:
    
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    x = 0
    innerbatch = []
    batches = []
    for row in csv_reader:
        if line_count >= 2:
            line_count += 1
            #1: Date
            #2: Symbol
            #3: Open
            #4: High
            #5: Low
            #6: Close
            #7: Volume
            temp = geminiData()
            
            temp.OpenPrice = float(row[3])
            temp.ClosePrice = float(row[6])
            temp.OpenPrice = int(temp.OpenPrice)
            temp.ClosePrice = int(temp.ClosePrice)
            # Gain loss
            
            temp.GainLoss = temp.OpenPrice - temp.ClosePrice
            # Variance
            temp.Variance = float(row[4]) - float(row[5])
            # Volume 
            temp.Volume = float( (row[7]))
            # Hour
            temp.Hour = row[1].split(" ")[1].split(":")[0]
            # Day
            temp.Day = row[1].split(" ")[0].split("-")[2]
            # Date YYYY-MM-DD ref: https://www.journaldev.com/23365/python-string-to-datetime-strptime
            datetime_str = row[1]

            datetime_object = datetime.strptime(datetime_str, '20%y-%m-%d %H:%M:%S')

             #print(datetime_object)  # printed in default format
            
            temp.Date = datetime_object
            geminiDatas.append(temp) 
            geminiX =  np.append( geminiX ,[[temp.GainLoss]], axis = 0)
            
            innerbatch.append(temp)

            if(temp.Hour == '01'):
                batches.append(innerbatch)
                innerbatch = []
                
                
        else:
            line_count += 1
            batches.append(innerbatch)

In [ ]:
geminiX.shape

In [ ]:
print(f'geminiX[t0:23]: {geminiX[20:43]} ,geminiX[t0+24] {geminiX[20+24]}' )

In [5]:
t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
Ts = t0 + np.arange(0., n_steps + 1) * resolution
ys = time_series(Ts)
return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

NameError: name 'batch_size' is not defined

In [36]:
n_iterations = 1500
batch_size = 50
t_min, t_max = 0, 30
n_steps = 20
resolution = 0.1

t0 = np.random.rand(batch_size, 1) * (t_max - t_min - n_steps * resolution)
Ts = t0 + np.arange(0., n_steps + 1) * resolution
ys = time_series(Ts)
y1 = ys[:, :-1].reshape(-1, n_steps, 1)
y2 =ys[:, 1:].reshape(-1, n_steps, 1)

rand = np.random.randint(batch_size)
t02 = geminiX[rand:rand+batch_size]
Ts2 = t02 + np.arange(21)
ys2 = geminiX[rand:rand+batch_size]

Data = []

for x in range(batch_size):
    rand = np.random.randint(len(geminiX))
    temp = np.empty([0,1])
    for i in range(n_steps+1):
        temp =  np.append(temp,[geminiX[rand+ 1]], axis = 0)
        i += 1
    print(f' temp: {temp.shape} ' )
    Data = np.append(Data, [[temp]], axis = 0)
    x += 1
    
        
#print(f't0: {t0.shape} , t02 {t02.shape}' )
#print(f'Ts: {Ts.shape} ,Ts2 {Ts2.shape}' )
#print(f'ys: {ys.shape} ,ys2 {ys2.shape}' )
#print(f' ts: {Ts2[8]} , ys {ys[1]}' )

print(f' Data: {Data.shape} , ys {ys.shape}' )

 temp: (21, 1) 


ValueError: all the input arrays must have same number of dimensions

In [10]:
t_min, t_max = 0, 30
resolution = 0.1

n_steps = 20
t_instance = np.linspace(12.2, 12.2 + resolution * (n_steps + 1), n_steps + 1)

def time_series(t):
    return t * np.sin(t) / 3 + 2 * np.sin(t*5)

def next_batch(batch_size, n_steps):
    
    rand = np.random.randint(len(geminiX))
    t0 = geminiX[rand:rand+batch_size]
    Ts = rand + np.arange(21)
    return ys[:, :-1].reshape(-1, n_steps, 1), ys[:, 1:].reshape(-1, n_steps, 1)

In [ ]:
#reset_graph()
tf.reset_default_graph()

n_steps = 20
n_inputs = 1
n_neurons = 100
n_outputs = 1

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_steps, n_outputs])

In [ ]:
cell = tf.contrib.rnn.OutputProjectionWrapper(
    tf.nn.rnn_cell.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu),
    output_size=n_outputs)

In [ ]:
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

In [ ]:
learning_rate = 0.001

loss = tf.reduce_mean(tf.square(outputs - y)) # MSE
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [ ]:
saver = tf.train.Saver()

In [ ]:
n_iterations = 1500
batch_size = 50

with tf.Session() as sess:
    init.run()
    for iteration in range(n_iterations):
        X_batch, y_batch = next_batch(batch_size, n_steps)
       # print(X_batch, "\t xy:", y_batch)
        sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if iteration % 100 == 0:
            mse = loss.eval(feed_dict={X: X_batch, y: y_batch})
            print(iteration, "\tMSE:", mse)
    
    saver.save(sess, "./my_time_series_model") # not shown in the book

In [ ]:
with tf.Session() as sess:                          # not shown in the book
    saver.restore(sess, "./my_time_series_model")   # not shown

    X_new = time_series(np.array(t_instance[:-1].reshape(-1, n_steps, n_inputs)))
    y_pred = sess.run(outputs, feed_dict={X: X_new})

In [ ]:
y_pred

In [ ]:
plt.title("Testing the model", fontsize=14)
#plt.plot(t_instance[:-1], time_series(t_instance[:-1]), "bo", markersize=10, label="instance")
#plt.plot(t_instance[1:], X_batch[0,:,0], "w*", markersize=10, label="target")
plt.plot(t_instance[1:], y_pred[0,:,0], "r.", markersize=10, label="prediction")
plt.legend(loc="upper left")
plt.xlabel("Time")

plt.show()